In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rnn
import lstm
import numpy as np
import theano
import theano.tensor as T
batch_size = 64
theano.config.optimizer='fast_compile'

In [ ]:
#model = rnn.Rnn(4, 20, 4, lr = 0.01, single_output=False, output_activation='nll', cost_function=T.nnet.softmax)
#model = rnn.RnnMiniBatch(4,20,4, 0.1, single_output=e_output=False, output_activation='nll', cost_function=T.nnet.softmax)
#model = lstm.Lstm(4,20,4, 0.1, single_output=False, output_activation='nll', cost_function=T.nnet.softmax)
model = lstm.LstmMiniBatch(4, 20, 4, 0.01, batch_size, single_output=False, output_activation='nll', cost_function=T.nnet.softmax)

In [ ]:
X_short_train = np.load('data/memory_task/short_output_data_test.npy')
Y_short_train = np.load('data/memory_task/short_output_data_train.npy')
X_short_train = np.load('data/running_max/short_input_data_train.npy')
Y_short_train = np.load('data/running_max/short_output_data_train.npy')
#Y_short_train = build_preds(X_short_train)

In [ ]:
print X_short_train[0],"\n\n",Y_short_train[0]

In [ ]:
def build_preds(X):
    Y = np.zeros(shape=(X.shape[0], X.shape[2]), dtype='float32')
    for idx, maxim in enumerate(np.argmax(np.sum(X, axis=1), axis=1)):
        Y[idx, maxim] = 1.
    return Y

def output(X):
    Y = np.zeros(shape=X.shape, dtype='float32')
    for idx, maxim in enumerate(np.argmax(X, axis=1)):
        Y[idx, maxim] = 1.
    return Y

In [ ]:
epochs = 10
#minibatch train
for epoch in range(epochs):
    loss = 0
    num_batch = len(X_short_train)/batch_size
    for i in range(num_batch):
        start_idx = i * batch_size
        end_idx = min((i+1) * (batch_size), len(X_short_train) -1)
        X_train = X_short_train[start_idx:end_idx]
        Y_train = Y_short_train[start_idx:end_idx]        
        #print X_train.shape, Y_train.shape, i, start_idx
        #print model.debug(X_train, Y_train)        
        loss += model.train(X_train, Y_train)
    print "epoch "+str(epoch) + " sum of batch loss "+str(loss)

In [ ]:
nb_epochs = 100
#naive sgd
for x in range(nb_epochs):
    loss = 0.
    for j in range(len(X_short_train)):  
        index = np.random.randint(0, len(X_short_train))
        X_train = X_short_train[index]
        Y_train = Y_short_train[index]        
        loss += model.train(X_train, Y_train)    
    print "epoch "+str(x)+ " error: "+str(loss)

In [ ]:
output(model.predictions(X_short_train[0:64]))